Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.59839300 0.271345993 0.00429035685 0.0039057243 1000
 beta   0.90473083 0.041329676 0.00065347955 0.0006065914 1000
sigma   5.10470219 0.193507310 0.00305961922 0.0027639030 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
alpha 154.0620000 154.41900000 154.593000 154.78000000 155.1290250
 beta   0.8240895   0.87680525   0.905077   0.93243725   0.9854926
sigma   4.7414180   4.97421750   5.098105   5.23073250   5.5067478



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.493 0.857796 5.58365; 154.632 0.89381 5.32403; … ; 154.603 0.941527 5.50939; 154.583 0.876703 4.74845]

Union{Missing, Float64}[154.336 0.869017 5.18194; 154.719 0.949946 5.17363; … ; 154.609 0.905822 5.15093; 154.76 0.92741 5.27213]

Union{Missing, Float64}[154.497 0.912111 5.16685; 154.734 0.901305 5.31143; … ; 154.341 0.929679 4.96868; 154.865 0.87624 5.22294]

Union{Missing, Float64}[154.819 0.936207 5.01359; 154.195 0.857868 5.22182; … ; 154.386 0.822411 5.32889; 153.811 0.873317 5.15803]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.59839300 0.271345993 0.00429035685 0.0039057243 1000
 beta   0.90473083 0.041329676 0.00065347955 0.0006065914 1000
sigma   5.10470219 0.193507310 0.00305961922 0.0027639030 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
alpha 154.0620000 154.41900000 154.593000 154.78000000 155.1290250
 beta   0.8240895   0.87680525   0.905077   0.93243725   0.9854926
sigma   4.7414180   4.97421750   5.098105   5.23073250   5.5067478



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*